<a href="https://colab.research.google.com/github/kdwang1808/2020-MCM/blob/master/M5_LGB_mem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 144568 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.21-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.21-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.21-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive
import os
import sys
os.chdir('drive/Colab Notebooks/M5_accuracy')

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import sys
import gc
import time
import lightgbm as lgb
# import graphviz
from  datetime import datetime, timedelta
from sklearn.metrics import mean_squared_error
from scipy.sparse import csr_matrix

In [0]:
train_start = 800
test_start = 1800
is_train = True
PRICE_DTYPES = {"store_id": "category", "item_id": "category", "wm_yr_wk": "int16","sell_price":"float16" }
CAL_DTYPES={"event_name_1": "category", "event_name_2": "category", "event_type_1": "category", 
      "event_type_2": "category", "weekday": "category", 'wm_yr_wk': 'int16', "wday": "int8",
      "month": "int8", "year": "int16", "snap_CA": "int8", 'snap_TX': 'int8', 'snap_WI': 'int8' }

start_day = train_start if is_train else test_start
numcols = [f"d_{day}" for day in range(start_day,1914)]
catcols = ['id', 'item_id', 'dept_id','store_id', 'cat_id', 'state_id']
SALE_DTYPES = {numcol:"float32" for numcol in numcols} 
SALE_DTYPES.update({col: "category" for col in catcols if col != "id"})

price = pd.read_csv('sell_prices.csv',dtype=PRICE_DTYPES)
cal = pd.read_csv('calendar.csv',dtype=CAL_DTYPES)
sale = pd.read_csv('sales_train_validation.csv',dtype=SALE_DTYPES,usecols=catcols+numcols) #usecols读取指定列
    
# 类别标签转换
for col, col_dtype in PRICE_DTYPES.items():
  if col_dtype == "category":
    price[col] = price[col].cat.codes.astype("int16")
    price[col] -= price[col].min()

cal["date"] = pd.to_datetime(cal["date"])
for col, col_dtype in CAL_DTYPES.items():
  if col_dtype == "category":
    cal[col] = cal[col].cat.codes.astype("int16")
    cal[col] -= cal[col].min()

for col in catcols:
  if col != "id":
    sale[col] = sale[col].cat.codes.astype("int16")
    sale[col] -= sale[col].min()

# 提交格式里有一部分为空
if not is_train:
  for day in range(1913+1, 1913+ 2*28 +1):
    sale[f"d_{day}"] = np.nan

sale = pd.melt(sale,
        id_vars = catcols,
        value_vars = [col for col in sale.columns if col.startswith("d_")],
        var_name = "d",
        value_name = "sales")
sale = sale.merge(cal, on= "d", copy = False)
sale = sale.merge(price, on = ["store_id", "item_id", "wm_yr_wk"], copy = False)
sale["dept_id"] = sale["dept_id"].astype("uint8")
sale["store_id"] = sale["store_id"].astype("uint8")
sale["cat_id"] = sale["cat_id"].astype("uint8")
sale["state_id"] = sale["state_id"].astype("uint8")
sale["sales"] = sale["sales"].astype("uint16")
sale["weekday"] = sale["weekday"].astype("uint8")
sale["wday"] = sale["wday"].astype("uint8")
sale["month"] = sale["month"].astype("uint8")
sale["event_name_1"] = sale["event_name_1"].astype("uint8")
sale["event_name_2"] = sale["event_name_2"].astype("uint8")
sale["event_type_1"] = sale["event_type_1"].astype("uint8")
sale["event_type_2"] = sale["event_type_2"].astype("uint8")

In [26]:
sale.head()
# sale.info()
# sale.sales.max()

,id,item_id,dept_id,store_id,cat_id,state_id,d,sales,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,HOBBIES_1_002_CA_1_validation,1,0,0,0,0,d_800,0,2013-04-07,11311,3,2,4,2013,0,0,0,0,1,1,0,3.970703
1,HOBBIES_1_002_CA_1_validation,1,0,0,0,0,d_801,0,2013-04-08,11311,1,3,4,2013,0,0,0,0,1,0,1,3.970703
2,HOBBIES_1_002_CA_1_validation,1,0,0,0,0,d_802,0,2013-04-09,11311,5,4,4,2013,0,0,0,0,1,1,1,3.970703
3,HOBBIES_1_002_CA_1_validation,1,0,0,0,0,d_803,0,2013-04-10,11311,6,5,4,2013,0,0,0,0,1,0,0,3.970703
4,HOBBIES_1_002_CA_1_validation,1,0,0,0,0,d_804,0,2013-04-11,11311,4,6,4,2013,0,0,0,0,0,1,1,3.970703


In [0]:
is_train=True
day=None
lags = [7, 28]
lag_cols = [f"lag_{lag}" for lag in lags ]

# 如果是测试集只需要计算一天的特征，减少计算量
# 注意训练集和测试集特征生成要一致
if is_train:
  for lag, lag_col in zip(lags, lag_cols):
    sale[lag_col] = sale[["id","sales"]].groupby("id")["sales"].shift(lag)
else:
  for lag, lag_col in zip(lags, lag_cols):
    sale.loc[sale.date == day, lag_col] = sale.loc[sale.date ==day-timedelta(days=lag), 'sales'].values  

# 将获取7天前的数据，28天前的数据做移动平均
wins = [7, 28]
if is_train:
  for win in wins :
    for lag,lag_col in zip(lags, lag_cols):
      sale[f"rmean_{lag}_{win}"] = sale[["id", lag_col]].groupby("id")[lag_col].transform(lambda x : x.rolling(win).mean())
else:
  for win in wins:
    for lag in lags:
      df_window = sale[(sale.date <= day-timedelta(days=lag)) & (sale.date > day-timedelta(days=lag+win))]
      df_window_grouped = df_window.groupby("id").agg({'sales':'mean'}).reindex(sale.loc[sale.date==day,'id'])
      sale.loc[sale.date == day,f"rmean_{lag}_{win}"] = df_window_grouped.sales.values   

# 处理时间特征
# 有的时间特征没有，通过datetime的方法自动生成
date_features = {"wday": "weekday",
         "week": "weekofyear",
         "month": "month",
         "quarter": "quarter",
         "year": "year",
         "mday": "day"}
for date_feat_name, date_feat_func in date_features.items():
  if date_feat_name in sale.columns:
    sale[date_feat_name] = sale[date_feat_name].astype("int16")
  else:
    sale[date_feat_name] = getattr(sale["date"].dt, date_feat_func).astype("int16")
sale["week"] = sale["week"].astype("uint8")
sale["quarter"] = sale["quarter"].astype("uint8")
sale["mday"] = sale["mday"].astype("uint8")
col_sales_feature = ["lag_7","lag_28","rmean_7_7","rmean_7_28","rmean_28_7","rmean_28_28"]
sale[col_sales_feature] = sale[col_sales_feature].astype("float16")
sale["wday"] = sale["wday"].astype("uint8")
sale["month"] = sale["month"].astype("uint8")

In [30]:
sale.head()
# sale.info()

,id,item_id,dept_id,store_id,cat_id,state_id,d,sales,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price,lag_7,lag_28,rmean_7_7,rmean_28_7,rmean_7_28,rmean_28_28,week,quarter,mday
0,HOBBIES_1_002_CA_1_validation,1,0,0,0,0,d_800,0,2013-04-07,11311,3,2,4,2013,0,0,0,0,1,1,0,3.970703,NaN,NaN,NaN,NaN,NaN,NaN,14,2,7
1,HOBBIES_1_002_CA_1_validation,1,0,0,0,0,d_801,0,2013-04-08,11311,1,3,4,2013,0,0,0,0,1,0,1,3.970703,NaN,NaN,NaN,NaN,NaN,NaN,15,2,8
2,HOBBIES_1_002_CA_1_validation,1,0,0,0,0,d_802,0,2013-04-09,11311,5,4,4,2013,0,0,0,0,1,1,1,3.970703,NaN,NaN,NaN,NaN,NaN,NaN,15,2,9
3,HOBBIES_1_002_CA_1_validation,1,0,0,0,0,d_803,0,2013-04-10,11311,6,5,4,2013,0,0,0,0,1,0,0,3.970703,NaN,NaN,NaN,NaN,NaN,NaN,15,2,10
4,HOBBIES_1_002_CA_1_validation,1,0,0,0,0,d_804,0,2013-04-11,11311,4,6,4,2013,0,0,0,0,0,1,1,3.970703,NaN,NaN,NaN,NaN,NaN,NaN,15,2,11


In [0]:
# 清洗数据，选择需要训练的数据
sale.dropna(inplace=True)
cat_feats = ['item_id', 'dept_id', 'store_id', 'cat_id', 'state_id'] + ["event_name_1", "event_name_2", "event_type_1", "event_type_2"]
useless_cols = ["id", "date", "sales","d", "wm_yr_wk", "weekday"]
train_cols = sale.columns[~sale.columns.isin(useless_cols)]
X_train = sale[train_cols]
y_train = sale["sales"]

In [39]:
X_train.head(100)

,item_id,dept_id,store_id,cat_id,state_id,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,1,0,0,0,0,2,4,2013,0,0,0,0,1,1,0,3.970703
1,1,0,0,0,0,3,4,2013,0,0,0,0,1,0,1,3.970703
2,1,0,0,0,0,4,4,2013,0,0,0,0,1,1,1,3.970703
3,1,0,0,0,0,5,4,2013,0,0,0,0,1,0,0,3.970703
4,1,0,0,0,0,6,4,2013,0,0,0,0,0,1,1,3.970703
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,17,0,0,0,0,7,4,2013,0,0,0,0,0,1,1,11.882812
96,18,0,0,0,0,2,4,2013,0,0,0,0,1,1,0,0.459961
97,18,0,0,0,0,3,4,2013,0,0,0,0,1,0,1,0.459961
98,18,0,0,0,0,4,4,2013,0,0,0,0,1,1,1,0.459961


In [35]:
y_train.head()

0    0
1    0
2    0
3    0
4    0
Name: sales, dtype: uint16